In [1]:
import numpy as np
import lightcones as lc
import lightcones.linalg as la
import matplotlib.pyplot as plt
from lightcones.linalg import mv
from lightcones import models
from lightcones.solvers.schrodinger import solve
plt.rcParams['text.usetex'] = True


In [2]:
# chain
n_sites = 15
# on-site energies
es = [1]*n_sites
# hoppings
hs = [0.05]*(n_sites-1)

# time grid
dt = 0.01
nt = 10000

# spread
spread = lc.spread(es, hs, nt, dt)

# rho_plus
rho_plus = lc.rho_plus(spread, dt)

# minimal light cone
rtol = 10**(-4)
ti_arrival, U_min, spread_min = lc.minimal_forward_frame(spread, rho_plus, dt, rtol)

In [3]:
# causal diamond frame

m = 3
spread_cd, U_cd = lc.causal_diamond_frame(spread_min, ti_arrival, U_min, rho_plus, dt, rtol, m)

In [ ]:
# moving frame

spread_mv, H_mv = lc.moving_frame(spread_cd, ti_arrival, U_cd, dt, m)